In [185]:
using Oscar

In [186]:
#define functions to solve f^* = min f(x) s.t. x in S(g)
#input: f: objective polynomials 
#       g: inequality constraints
#       h:= (g_j-y_j^2)
#       R: ring of poynomials
#       n: number of variables
#       m: number of inequality constraints
#       x: vector of variables
#       lamb: vector of multipliers
#       t: scalar variable
#       radical_comp: compute radical

#output: system of univariate polynomials whose common set of real roots contains f^*

function solve_FJ(n,m,f,g,h,R,x,lamb,t;radical_comp=true)

@time begin
    
#compute the set h_FJ of polynomials from Fritz John conditions
h_FJ=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_FJ[i]=lamb[1]*derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_FJ[i]-=lamb[j+1]*derivative(g[j], i)
    end
end

for j in 1:m
    h_FJ=[h_FJ;lamb[j+1]*g[j]]
end

h_FJ=[h_FJ;1-sum(lamb.^2)]
h=[h;h_FJ]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)

if radical_comp
    RI = radical(ideal(R, h))#compute radical of V(h)
else
    RI = ideal(R, h)
end
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+2,n+2*m+2))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

println(h_uni)# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_FJ (generic function with 1 method)

In [187]:
function solve_FJ_plus(n,m,f,g,h,R,x,lamb,t)

@time begin
    
#compute the set h_FJ of polynomials from Fritz John conditions
h_FJ=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_FJ[i]=lamb[1]^2*derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_FJ[i]-=lamb[j+1]^2*derivative(g[j], i)
    end
end

for j in 1:m
    h_FJ=[h_FJ;lamb[j+1]^2*g[j]]
end

h_FJ=[h_FJ;1-sum(lamb.^2)]
h=[h;h_FJ]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)
    
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+2,n+2*m+2))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

println(h_uni)# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_FJ_plus (generic function with 1 method)

In [188]:
function solve_KKT(n,m,f,g,h,R,x,lamb,t)

@time begin
    
#compute the set h_KKT of polynomials from Karush–Kuhn–Tucker conditions
h_KKT=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_KKT[i]=derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_KKT[i]-=lamb[j]*derivative(g[j], i)
    end
end

for j in 1:m
    h_KKT=[h_KKT;lamb[j]*g[j]]
end

h=[h;h_KKT]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)
    
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
#println(G)
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+1,n+2*m+1))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

h_uni# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_KKT (generic function with 1 method)

In [189]:
function solve_KKT_plus(n,m,f,g,h,R,x,lamb,t)

@time begin
    
#compute the set h_KKT of polynomials from Karush–Kuhn–Tucker conditions
h_KKT=Vector{fmpq_mpoly}(undef,n)
for i in 1:n
    h_KKT[i]=derivative(f,i)
end
for j in 1:m
    for i in 1:n
        h_KKT[i]-=lamb[j]^2*derivative(g[j], i)
    end
end

for j in 1:m
    h_KKT=[h_KKT;lamb[j]^2*g[j]]
end

h=[h;h_KKT]

h=[h;t[1]-f]#h=(gj(x)-yj^2,h_FJ,x_{n+1}-f)
    
RI = radical(ideal(R, h))#compute radical of V(h)
#RI = ideal(R, h)
#gens(RI)
    
G=groebner_basis(RI, ordering=lex(gens(R)))
    
#println(G)
    
check=1
alpha=zeros(Int64,1)
h_uni=Vector{fmpq_mpoly}([])
for j=1:length(G)
    check=1
    for i in 1:length(G[j])
        alpha=exponent_vector(G[j],i) 
        if sum(alpha[t] for t in setdiff(1:n+2*m+1,n+2*m+1))>0
            check=0
        end
    end
    if check==1
        h_uni=[h_uni;G[j]]
    end
end

h_uni# generators of the image f(S(g)\cap V(h_FJ))
    
end
    
end

solve_KKT_plus (generic function with 1 method)

In [190]:
#Example 1 in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[2]#objective polynomial
g=[-x[1];x[1]-x[2]^2]#inequality constraints


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[t_{1}]
  0.005381 seconds (2.00 k allocations: 72.406 KiB)


In [191]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [192]:
#Example 2 in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))

f=x[1]-5*x[2]#objective polynomial

g=[x[1]^2-x[2];-x[1]^2+4*x[2];-x[2]+1]#inequality constraints


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.014456 seconds (39.43 k allocations: 869.531 KiB)


1-element Vector{fmpq_mpoly}:
 100*t_{1}^6 + 1975*t_{1}^5 + 14001*t_{1}^4 + 41395*t_{1}^3 + 39295*t_{1}^2 - 12150*t_{1} + 504

In [193]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 100*u^6 + 1975*u^5 + 14001*u^4 + 41395*u^3 + 39295*u^2 - 12150*u + 504

r = roots(m,isolate_real = true)

6-element Vector{acb}:
 [-7.00000 +/- 3.20e-6]
 [-6.00000 +/- 4.11e-6]
 [-4.00000 +/- 1.24e-6]
 [-3.000000 +/- 5.76e-7]
 [0.050000000 +/- 4.06e-10]
 [0.200000000 +/- 9.30e-10]

In [194]:
#[11, Example 3] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=(x[1]+1)^2+x[2]^2-1

g=[x[1]^3-x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t,radical_comp=true)

fmpq_mpoly[27*t_{1}^4 + 59*t_{1}^3 + 60*t_{1}^2 + 28*t_{1}]
  0.008932 seconds (10.96 k allocations: 241.109 KiB)


In [195]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 27*u^4 + 59*u^3 + 60*u^2 + 28*u

r = roots(m)

4-element Vector{acb}:
 [-1.0000000 +/- 5.42e-9]
 [+/- 3.16e-29]
 [-0.5925926 +/- 1.65e-8] + [0.8281733 +/- 3.42e-8]*im
 [-0.5925926 +/- 1.46e-8] + [-0.8281733 +/- 3.26e-8]*im

In [196]:
#The case t=-1
#Finding x_1^*

using Oscar

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[1]

g=[x[1]^3-x[2]^2;(x[1]+1)^2+x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t,radical_comp=true)

fmpq_mpoly[t_{1}^5 + 2*t_{1}^4 + 3*t_{1}^3 + 3*t_{1}^2 + t_{1}]
  0.017011 seconds (24.11 k allocations: 412.806 KiB)


In [197]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u^5 + 2*u^4 + 3*u^3 + 3*u^2 + u

r = roots(m)# containts x_1^*

5-element Vector{acb}:
 [-1.000000 +/- 1.10e-8]
 [-0.56984029 +/- 7.62e-9]
 [+/- 1.02e-27]
 [-0.2150799 +/- 5.33e-8] + [1.30714128 +/- 8.79e-9]*im
 [-0.2150799 +/- 5.36e-8] + [-1.30714128 +/- 9.02e-9]*im

In [198]:
#The case t=-1
#Finding x_2^*

using Oscar

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[2]

g=[x[1]^3-x[2]^2;(x[1]+1)^2+x[2]^2]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t,radical_comp=true)

fmpq_mpoly[t_{1}^7 - 2*t_{1}^5 + 5*t_{1}^3 + t_{1}]
  0.017537 seconds (28.43 k allocations: 493.865 KiB)


In [199]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u^7 - 2*u^5 + 5*u^3 + u

r = roots(m)#containts x_2^*

7-element Vector{acb}:
 [+/- 4.31e-25]
 [+/- 1.79e-9] + [-0.43015971 +/- 2.73e-9]*im
 [+/- 1.79e-9] + [0.43015971 +/- 2.73e-9]*im
 [-1.3071413 +/- 3.54e-8] + [0.7849201 +/- 5.93e-8]*im
 [-1.3071413 +/- 3.54e-8] + [-0.7849201 +/- 5.93e-8]*im
 [1.3071413 +/- 3.54e-8] + [-0.7849201 +/- 5.93e-8]*im
 [1.3071413 +/- 3.54e-8] + [0.7849201 +/- 5.93e-8]*im

In [200]:
#The case t=0
#Finding x_1^*

using Oscar

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[1]

g=[x[1]^3-x[2]^2;(x[1]+1)^2+x[2]^2-1]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t,radical_comp=true)

fmpq_mpoly[t_{1}^4 + 3*t_{1}^3 + 4*t_{1}^2 + 4*t_{1}]
  0.060131 seconds (26.70 k allocations: 1.191 MiB)


In [201]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u^4 + 3*u^3 + 4*u^2 + 4*u

r = roots(m)

4-element Vector{acb}:
 [-2.00000000 +/- 3.73e-9]
 0
 [-0.50000000 +/- 6.13e-9] + [1.3228757 +/- 5.03e-8]*im
 [-0.50000000 +/- 2.48e-9] + [-1.32287566 +/- 6.76e-9]*im

In [202]:
#The case t=0
#Finding x_2^*

using Oscar

n=2
m=2

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[2]

g=[x[1]^3-x[2]^2;(x[1]+1)^2+x[2]^2-1]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ(n,m,f,g,h,R,x,lamb,t,radical_comp=true)

fmpq_mpoly[t_{1}^7 - 6*t_{1}^5 + 13*t_{1}^3 - 8*t_{1}]
  0.015154 seconds (27.28 k allocations: 549.917 KiB)


In [203]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u^7 - 6*u^5 + 13*u^3 - 8*u

r = roots(m)

7-element Vector{acb}:
 [-1.000000 +/- 1.03e-8]
 [+/- 3.04e-28]
 [1.000000 +/- 1.03e-8]
 [-1.6322419 +/- 4.52e-8] + [0.4052327 +/- 5.40e-8]*im
 [-1.6322419 +/- 4.29e-8] + [-0.4052327 +/- 5.21e-8]*im
 [1.6322419 +/- 4.29e-8] + [0.4052327 +/- 5.21e-8]*im
 [1.6322419 +/- 4.43e-8] + [-0.4052327 +/- 5.29e-8]*im

In [204]:
#[11, Example 5] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))

f=x[1]-x[2]

g=[(x[1]-x[2])^3]


h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ_plus(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[t_{1}]
  0.032922 seconds (2.77 k allocations: 95.891 KiB)


In [205]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u

r = roots(m)

1-element Vector{acb}:
 0

In [206]:
#[6, Example A2] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))

f=(x[1]^2+x[2]^2-2)*(x[1]^2+x[2]^2)#objective polynomial

g=[(x[1]^2+x[2]^2-2)*(x[1]-3)]#inequality constraints

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.061127 seconds (17.71 k allocations: 785.406 KiB)


1-element Vector{fmpq_mpoly}:
 t_{1}^3 - 62*t_{1}^2 - 63*t_{1}

In [207]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = u^3 - 62*u^2 - 63*u

r = roots(m,isolate_real = true)

3-element Vector{acb}:
 [-1.00000000 +/- 1.40e-9]
 [+/- 6.08e-29]
 [63.00000000 +/- 1.40e-9]

In [208]:
#[11, Example 11] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=3
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))

eps=1//10

f=x[1]^4*x[2]^2+x[1]^2*x[2]^4+x[3]^6-3*x[1]^2*x[2]^2*x[3]^2+eps*(x[1]^2+x[2]^2+x[3]^2) 
g=[1-x[1]^2-x[2]^2-x[3]^2]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.172232 seconds (1.43 M allocations: 31.758 MiB)


1-element Vector{fmpq_mpoly}:
 49207500000000000*t_{1}^11 - 81961242187500000*t_{1}^10 + 35401891640625000*t_{1}^9 - 5747811890625000*t_{1}^8 + 576144305156250*t_{1}^7 - 50773802062500*t_{1}^6 + 2487391238125*t_{1}^5 - 34983679000*t_{1}^4 + 1586740350*t_{1}^3 - 4087040*t_{1}^2 + 182336*t_{1}

In [209]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 49207500000000000*u^11 - 81961242187500000*u^10 + 35401891640625000*u^9 - 5747811890625000*u^8 + 576144305156250*u^7 - 50773802062500*u^6 + 2487391238125*u^5 - 34983679000*u^4 + 1586740350*u^3 - 4087040*u^2 + 182336*u

r = roots(m,isolate_real = true)

11-element Vector{acb}:
 [+/- 1.22e-11]
 [0.100000 +/- 2.57e-8]
 [0.1156250 +/- 3.35e-8]
 [0.3500000 +/- 2.03e-8]
 [1.1000000 +/- 1.89e-8]
 [+/- 1.82e-10] + [0.012171612 +/- 5.68e-10]*im
 [+/- 1.22e-10] + [-0.012171612 +/- 5.12e-10]*im
 [+/- 3.29e-10] + [0.024343225 +/- 5.47e-10]*im
 [+/- 3.29e-10] + [-0.024343225 +/- 5.47e-10]*im
 [+/- 1.32e-9] + [0.09737290 +/- 2.18e-9]*im
 [+/- 1.35e-9] + [-0.09737290 +/- 2.21e-9]*im

In [210]:
#[11, Example 12] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))


f=x[1]*x[2]+x[1]^3+x[2]^3
g=[x[1];x[2];1-x[1]-x[2]]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT(n,m,f,g,h,R,x,lamb,t)

  0.017377 seconds (20.35 k allocations: 447.312 KiB)


1-element Vector{fmpq_mpoly}:
 54*t_{1}^4 - 83*t_{1}^3 + 30*t_{1}^2 - t_{1}

In [211]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 54*u^4 - 83*u^3 + 30*u^2 -u

r = roots(m,isolate_real = true)

4-element Vector{acb}:
 0
 [0.037037037 +/- 1.73e-10]
 [0.50000000 +/- 3.52e-9]
 1.0000000000000000000

In [212]:
#[11, Example 13] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=3
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m), "t" => (1:1))


f=x[1]^4*x[2]^2+x[2]^4*x[3]^2+x[3]^4*x[1]^2-3*x[1]^2*x[2]^2*x[3]^2
g=[1-x[1]^2;1-x[2]^2;1-x[3]^2]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_KKT_plus(n,m,f,g,h,R,x,lamb,t)

  1.235998 seconds (724.44 k allocations: 14.607 MiB)


1-element Vector{fmpq_mpoly}:
 64*t_{1}^4 - 271*t_{1}^3 + 234*t_{1}^2 - 27*t_{1}

In [213]:
CC = ComplexField(64)
C, u = PolynomialRing(CC, "u")

m = 64*u^4 - 271*u^3 + 234*u^2 - 27*u

r = roots(m,isolate_real = true)

4-element Vector{acb}:
 [+/- 6.43e-26]
 [0.13616744 +/- 3.33e-9]
 [1.0000000 +/- 6.40e-9]
 [3.09820756 +/- 7.77e-9]

In [214]:
#[12, Example 5] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=3

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))


f=x[1]+x[2]
g=[x[1]^3;x[2]^3;-x[1]*x[2]]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ_plus(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[]
  0.135581 seconds (80.10 k allocations: 4.897 MiB)


In [215]:
#[11, Example 9] in Table 1 of ``A symbolic algorithm for exact polynomial optimization strengthened with Fritz John conditions'' by N. H. A. Mai
using Oscar

n=2
m=1

R, x, y, lamb, t = PolynomialRing(QQ, "x" => (1:n), "y" => (1:m), "lamb" => (1:m+1), "t" => (1:1))


f=x[1]
g=[x[1]*x[2]^2-1]

h=[g[j]-y[j]^2 for j=1:m]#transform to equality constraints

solve_FJ_plus(n,m,f,g,h,R,x,lamb,t)

fmpq_mpoly[1]
  0.002953 seconds (1.30 k allocations: 43.758 KiB)
